<a href="https://colab.research.google.com/github/chsachin799/100-days-of-ML/blob/main/29_ML_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df = pd.read_csv('train.csv')


In [ ]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

#train test split

x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)


In [ ]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [ ]:
#Step1 :imputation transformer

trf1 = ColumnTransformer([
    ('impute age',SimpleImputer(),[2]),
('impute embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [ ]:
#one Hot Encoding

trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [ ]:
#scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10)) #iss range me jitne columns hai sab select karta hai aur uspe minmaxscaler implemment karta hai

])

In [ ]:
#feature scaling
trf4 = SelectKBest(score_func=chi2,k=8)  # 8 represents top 5 most important cols will be selected.

In [ ]:
trf5 = DecisionTreeClassifier()

# Create Pipeline

In [ ]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),    #list of tuples with object and theri names.
    ('trf4',trf4),    #this is what a pipeline object is.
    ('trf5',trf5)
])

# pipeline vs make_pipeline

we can use make_pipeline for simplicty as we dont need to pass names for the object. So it better to use make_pipeline than pipeline itself.

In [ ]:
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)


In [ ]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute age', SimpleImputer(),
                                                  [2]),
                                                 ('impute embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f4b952a77e0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [ ]:
#display pipeline
from sklearn import set_config
set_config(display='diagram')

In [ ]:
y_pred = pipe.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

# Cross Validation using pipeline

In [ ]:
from sklearn.model_selection import cross_val_score
grid = cross_val_score(pipe,x_train,y_train,cv=5,scoring='accuracy').mean() # here it will take accuracy for 5 times and will calculate its mean at the end (if used .mean())and returns mean accuracy score


# GridSearch using Pipeline

In [ ]:
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(x_train,y_train)

ValueError: Invalid parameter 'trf5' for estimator Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute age', SimpleImputer(),
                                                  [2]),
                                                 ('impute embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformer